# Exploration 4. 멋진 작사가 만들기

### Step1. 데이터 다운로드

#### ~/aiffel/lyricist/data/lyrics 에 데이터가 이미 존재한다.

### Step2. 데이터 읽어오기

##### 자연어 데이터를 코퍼스(corpus)라고 부르며 특정 도메인으로부터 수집된 텍스트 집합을 말한다.
##### 텍스트 데이터의 파일형식은 txt, csv, xml파일 등으로 다양한데 이번 프로젝트에서는 txt 파일을 가져와 사용한다.

In [1]:
import glob
import os, re
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt파일을 모두 읽어서 raw_corpus에 담는다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Example: \n", raw_corpus[:3])

데이터 크기: 187088
Example: 
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


### Step3. 데이터 정제

##### 자연어 처리에 있어서 텍스트를 용도에 맞게 사전에 처리하는 작업인 텍스트 전처리가 매우 중요한 작업이다.
##### 정규 표현식에서 re.sub()함수를 사용하여 정규 표현식과 패턴과 일치하는 문자열을 찾아 다른 문자열로 대체할 수 있다.

In [2]:
# 정규표현식(Regex)

# 입력된 문장을
#  1. 소문자로 바꾸고, 양쪽 공백을 지운다.
#  2. 특수문자 양쪽에 공백을 넣고
#  3. 여러개의 공백은 하나의 공백으로 바꾼다.
#  4. a-zA-Z?.!가 아닌 모든 문자를 하나의 공백으로 바꾼다.
#  5. 다시 양쪽 공백을 지웁니다.
#  6. 문장 시작에는 <start>, 끝에는 <end>를 추가한다.
# 위의 순서대로 처리해주면 문제가 되는 상황을 방지할 수 있다.

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 (이미 숫자도 제외되고 있음. ^기호가 대괄호 안에 있으면 대괄호 안에 있는 문자 제외하고 공백으로 처리)
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence(raw_corpus[5]))

<start> none were level on the mind <end>


In [3]:
# 여기에 정제된 문장을 모읍시다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    # 정제를 하고 담아주세요
    
    if len(sentence) == 0: continue
    tmp = preprocess_sentence(sentence)
    if len(tmp.split()) > 15: continue     # 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외
    corpus.append(tmp)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

##### 케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공한다. 
##### tokenizer.fit_on_texts() 안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.
##### tokenizer.texts_to_sequences()로 텍스트 시퀀스의 모든 단어들을 각 정수로 맵핑한다.
##### 문장의 길이가 각각 다 다르기 때문에 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업을 패딩(padding)이라고 한다.
##### 문장의 길이를 가장 긴 문장에 맞추는 것이 아닌 maxlen=15로 인자를 정해서 모든 문서의 길이를 동일하게 맞춰준다.

In [4]:
# 토큰화할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용한다.

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  # pre-padding을 적용할 경우 성능이 더 좋아진다~!!!뒤로 가면 갈수록 데이터가 더 중요하기 때문에
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 117   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 635 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f9f15cb1b50>


In [5]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 
print(tensor[:3, :10])

[[   2   62  271   27   94  546   20   86  743   90]
 [   2  117    6 6269   10    6 2310    3    0    0]
 [   2   62   17  102  184 2718    3    0    0    0]]


In [6]:
# tokenizer에 구축된 단어 사전이 어떻게 구축되었는지 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2  62 271  27  94 546  20  86 743  90   3   0   0   0]
[ 62 271  27  94 546  20  86 743  90   3   0   0   0   0]


### Step4. 평가 데이터셋 분리

In [8]:
# 데이터셋 객체 생성

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듦
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
# 훈련데이터와 평가데이터 분리

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state= 10)
print("Source Train:", enc_train.shape)
print("Source Val:", enc_val.shape)

Source Train: (124981, 14)
Source Val: (31246, 14)


### Step5. 인공지능 만들기

In [10]:
# 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성한 구조도

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 3000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)   

In [11]:
# 모델 학습시키기

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction = 'none')

model.compile(loss = loss, optimizer = optimizer)
model.fit(enc_train, dec_train, epochs = 5, validation_data=(enc_val, dec_val))


Epoch 1/5
3906/3906 [==============================] - 2019s 503ms/step - loss: 3.1878 - val_loss: 2.6402
Epoch 2/5
3906/3906 [==============================] - 1964s 503ms/step - loss: 2.4026 - val_loss: 2.3777
Epoch 3/5
3906/3906 [==============================] - 1964s 503ms/step - loss: 1.8861 - val_loss: 2.2380
Epoch 4/5
3906/3906 [==============================] - 1964s 503ms/step - loss: 1.4856 - val_loss: 2.1942
Epoch 5/5
3906/3906 [==============================] - 1964s 503ms/step - loss: 1.2356 - val_loss: 2.2221


In [12]:
# 평가하기

def generate_text(model, tokenizer, init_sentence = "<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장을 만듭니다.
    #  1. 입력받은 문장의 텐서를 입력합니다.
    #  2. 예측된 값 중 가장 높을 확률인 word index를 뽑아냅니다.
    #  3. 2에서 예측된 word index를 문장 뒤에 붙입니다.
    #  4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다.
    
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token:break
        if test_tensor.shape[1] >= max_len:break
            
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

generate_text(model, tokenizer, init_sentence = "<start> i love", max_len=20)

'<start> i love you , bye bye <end> '

# 회고

1. NLP를 처음 접해보았는데 데이터를 정제하는 부분이 새롭기도 하고 언어를 사용하고자 하는 용도에 맞게 처리하는 것이 어려우면서도 중요한 작업인 걸 깨달았다.      
     
          
2. NLP를 접하면서 아주 조금은 CNN과 RNN의 개념이 구분이 가기 시작했다. CNN은 feature(특징)을 추출하여 패턴을 파악하는 구조이며 RNN은 시퀀스 구조로 순차적으로 이루어진 데이터 처리에 적합하며 과거의 값이 현재의 값에 반영이 되는 구조이다.     
      
           
3. val_loss를 2.2이하 수준으로 줄이기 위해 embedding_size를 256, 512, 1024로 변화를 주었고 hidden_size로 2000, 3000으로 변화를 주었지만 2.2 ~ 2.3 수준으로 크게 변화가 있지는 않았다.      
     
          
4. TF 마스터 시간에 padding에 대해서 이야기를 나누었는데 가변적인 문장 길이를 같은 길이로 맞춰주기 위해서 사용된다는 것을 알게 되었고 뒤에서 채우는 post-padding보다는 앞에서부터 패딩을 채우는 pre-padding이 더 성능이 좋게 나올 것이라는 이야기를 나누었다. 그래서 실제로 padding의 default값이 'pre'로 되어있다는 이야기도 나누었다. 실제로 padding을 'pre'로 넣고 다시 코드를 실행해 보았지만 val_loss가 더 낮게 나올것이라는 내 예상과는 달리 그렇게 성능이 좋게 나오지는 않았다. (padding='pre'를 넣고 돌린 결과 val_loss가 2.2 ~ 2.3으로 padding='post'와 별반 다르지 않았다.)    
      
           
5. epoch당 20~30분 정도 시간이 걸려 val_loss를 도출하기까지 꽤 오랜 시간이 걸려 많이 돌려보지 못하였다. 3번정도 코드를 실행해 보았을 때의 결과는 다음과 같이 나왔다. input으로 '<start> i love'만 입력하였는데 뒤에 자동으로 문장이 완성되는 것이 신기했다. 다음에 시간있을 때 다른 문장도 한 번 넣어서 만들어 봐야겠다.       
         
- 1회차 : 'i love you so much, i love you so much'        
- 2회차 : 'i love the way you shake your thing'      
- 3회차 : 'i love you, bye bye'      
